In [22]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from glob import glob
from numpy import array
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import statistics

In [23]:
def plot_usecase_json(experiment):
    res = []
    for pth in ["neo4j","blazegraph","virtuoso","graphdb","kitt"]:#,"jena"
        for file in glob('{}/{}.*.json'.format(pth, experiment)):
            #print(file)
            parts = file.split('.')
            run = '.'.join(parts[2:-1])
            with open(file, 'rt') as fi:
                dat=json.load(fi)
                for r in dat:
                    res.append({
                        "query_id":r['query'],
                        "query_idc":str(r['query']),
                        "size":int(parts[-2]),
                        "parallel":1,
                        "version":'.'.join(parts[3:6]),
                        "database":pth,
                        "bench":experiment,
                        "run":run,
                        "executecount":r.get('executecount',len(r['result_size'])),
                        "aqet":0,
                        "time_min":min(r['times']) if r['times'] else 0,
                        "time_avg":sum(r['times'])/len(r['times']) if r['times'] else 0,
                        "time_med":statistics.median(r['times']) if r['times'] else 0,
                        "time_max":max(r['times']) if r['times'] else 0,
                        "minresults":min(r['result_size']) if r['result_size'] else 0,
                        "maxresults":sum(r['result_size'])/len(r['result_size']) if r['result_size'] else 0,
                        "results_med":statistics.median(r['result_size']) if r['result_size'] else 0,
                        "avgresults":min(r['result_size']) if r['result_size'] else 0,
                        "timeoutcount":0
                    })
    return pd.DataFrame(res)


def plot_usecase_xml():
    res = []
    for pth in ["neo4j"]:#,"jena"
        for file in glob(f'{pth}/bsbm.explore.*.json'):
            #print(file)
            parts = file.split('.')
            run = '.'.join(parts[2:-1])
            with open(file, 'rt') as fi:
                dat=json.load(fi)
                for r in dat:
                    res.append({
                        "query_id":r['query'],
                        "query_idc":str(r['query']),
                        "size":int(parts[-2]),
                        "parallel":1,
                        "version":'.'.join(parts[3:6]),
                        "database":pth,
                        "bench":"bsbm",
                        "run":run,
                        "executecount":r.get('executecount',len(r['result_size'])),
                        "aqet":0,
                        #"qps":r.get('qps',int(parts[-2])/r['time_avg']),
                        "time_min":min(r['times'])/1000.0,
                        "time_avg":sum(r['times'])/len(r['times'])/1000.0,
                        "time_max":max(r['times'])/1000.0,
                        "minresults":min(r['result_size']),
                        "maxresults":sum(r['result_size'])/len(r['result_size']),
                        "avgresults":min(r['result_size']),
                        "timeoutcount":0
                    })
    for pth in ["blazegraph","graphdb","virtuoso"]:
        for file in glob('{}/bsbm.{}.*.xml'.format(pth,'explore')):
            #print(file)
            parts = file.split('.')
            run = '.'.join(parts[2:-1])
            #print(parts, run)
            for query in ET.parse(file).getroot().find('queries').findall('query'):
                query_id = int(query.attrib['nr'])
                for child in query.iter():
                    if child.tag == "aqet":
                        pass
                    elif child.tag == "avgresults":
                        res.append({
                            "query_id":query_id,
                            "query_idc":str(query_id),
                            "size":int(parts[-3]),
                            "parallel":int(parts[-2]),
                            "version":'.'.join(parts[3:6]),
                            "database":parts[2],
                            "bench":"bsbm",
                            "run":run,
                            "executecount":float(query.find('executecount').text),
                            "qps":float(query.find('qps').text),
                            "aqet":float(query.find('aqet').text),
                            "time_avg":float(query.find('aqetg').text),
                            "time_min":float(query.find('minqet').text),
                            "time_max":float(query.find('maxqet').text),
                            "minresults":float(query.find('minresults').text),
                            "maxresults":float(query.find('maxresults').text),
                            "avgresults":float(query.find('avgresults').text),
                            "timeoutcount":float(query.find('timeoutcount').text)
                        })
    for pth in ["kitt"]:
        for file in glob(f'{pth}/bsbm.explore.kitt.*.json'):
            parts = file.split('.')
            run = '.'.join(parts[2:-1])
            with open(file, 'rt') as fi:
                dat=json.load(fi)
                for r in dat:
                    res.append({
                        "query_id":int(r['query'].replace("q","").replace("a","").replace("b","")),
                        "query_idc":str(r['query']).replace("q","").replace("a","").replace("b",""),
                        "size":int(parts[-3]),
                        "parallel":int(parts[-2]),
                        "version":'.'.join(parts[3:6]),
                        "database":parts[2],
                        "bench":"bsbm",
                        "run":run,
                        "executecount":r.get('executecount',len(r['resizes'])),
                        "aqet":0,
                        #"qps":r.get('qps',int(parts[-2])/r['time_avg']),
                        "time_min":r['time_min'],
                        "time_avg":r['time_avg'],
                        "time_max":r['time_min'],
                        "minresults":r['minresults'],
                        "maxresults":r['maxresults'],
                        "avgresults":r['avgresults'],
                        "timeoutcount":0
                    })
    return pd.DataFrame(res)

In [243]:
df0 = plot_usecase_xml()
df0['size'] = df0['size'] / 1000.0
df0 = df0.sort_values(by=["size","query_id"])
#df0 = df0[df0['size']!=1000]
df0['time_avg'][df0['size']==1.0]=0
df1 = plot_usecase_json("lubm")
df1['time_avg'] = df1['time_avg'] / 1000.0
df2 = plot_usecase_json("factory")
df2['time_avg'] = df2['time_avg'] / 1000.0
df2['bench'] = "dtbm"
df = df0.append(df1).append(df2)
df = df[df['size']!=1.0]
df

/var/folders/2b/7vzpxk7n7xxbc61g6nrl55tr0000gn/T/ipykernel_96194/648927977.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,query_id,query_idc,size,parallel,version,database,bench,run,executecount,aqet,time_min,time_avg,time_max,minresults,maxresults,avgresults,timeoutcount,qps,time_med,results_med
12,1,1,2.0,1,4.4.5,neo4j,bsbm,neo4j.4.4.5.2000,500.0,0.000000,0.013587,0.015741,0.387209,1.0,1.0,1.0,0.0,NaN,NaN,NaN
127,1,1,2.0,16,2.2.0,blazegraph,bsbm,blazegraph.2.2.0.2000.16,500.0,0.037970,0.016602,0.036807,0.152213,0.0,10.0,0.5,0.0,413.83,NaN,NaN
171,1,1,2.0,16,9.10.0,graphdb,bsbm,graphdb.9.10.0.2000.16,500.0,0.006376,0.001244,0.005833,0.035745,0.0,10.0,0.5,0.0,2412.79,NaN,NaN
204,1,1,2.0,16,7.2.2,virtuoso,bsbm,virtuoso.7.2.2.2000.16,500.0,0.013962,0.004607,0.013366,0.027007,0.0,10.0,0.5,0.0,1125.24,NaN,NaN
295,1,1,2.0,16,3.1.0,kitt,bsbm,kitt.3.1.0.2000.16,500.0,0.000000,0.001851,0.003330,0.001851,0.0,10.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,8,8,2.0,1,0.2.json,kitt,dtbm,1.0.2,50.0,0.000000,29.413700,0.030474,32.432556,1000.0,1000.0,1000.0,0.0,NaN,30.353904,1000.0
356,9,9,2.0,1,0.2.json,kitt,dtbm,1.0.2,50.0,0.000000,21.270037,0.025354,27.169466,1.0,1.0,1.0,0.0,NaN,25.633812,1.0
357,10,10,2.0,1,0.2.json,kitt,dtbm,1.0.2,50.0,0.000000,44.264555,0.050642,55.842638,1.0,1.0,1.0,0.0,NaN,50.764084,1.0
358,11,11,2.0,1,0.2.json,kitt,dtbm,1.0.2,50.0,0.000000,14.290094,0.015240,16.248703,1.0,1.0,1.0,0.0,NaN,15.282273,1.0


In [244]:
fig=px.bar(data_frame=df, x='database', y='time_avg', color='query_idc', facet_col='size', facet_row='bench',facet_row_spacing=0.04, facet_col_spacing=0.022,
    labels = {'database': '', 'time_avg': '', 'query_idc': 'Query', 'bench': ''}, template = 'plotly_white')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", xanchor="right", x=1, y=1.05))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(matches=None, showticklabels=True)
fig.update_traces(marker_line_width=0)
fig.layout.yaxis7.title.text = 'response time in seconds'
fig.layout.xaxis3.title.text = 'backend type'
fig

In [245]:
#fig.write_image("all_time_avg.pdf", width=1200, height=500)

In [246]:
fig=px.bar(data_frame=df, x='database', y='time_avg', color='query_idc', facet_col='size', facet_row='bench',facet_row_spacing=0.04, facet_col_spacing=0.04,
    labels = {'database': '', 'time_avg': '', 'query_idc': 'Query', 'bench': ''}, log_y=True, template = 'plotly_white')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", xanchor="right", x=1, y=1.05))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(matches=None, showticklabels=True)
fig.update_traces(marker_line_width=0)
fig.layout.yaxis7.title.text = 'response time in seconds'
fig.layout.xaxis3.title.text = 'backend type'
fig

In [17]:
#fig.write_image("all_time_avg_log.pdf", width=1200, height=500)